# Top End Assembly Vibration Study-- MTCamHexapod Telemetries

##### As described in OBS-467, we have been noticing strong vibrations on the Top End Assembly (TEA) of the Simonyi Survey Telescope (SST). The first time we saw these vibrations was on March 9th (Saturday) and on March 10th (Sunday).

##### Accordingly to SITCOM-1285, on March 15th (Friday), we ran soak tests using the Rotator at different configurations. We want to investigate any possible vibrations during that period to confirm the hypothesis that the Rotator could be causing this vibration.



In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time
import warnings

from argparse import Namespace
from astropy.time import Time, TimeDelta
from scipy.fft import fft, fftfreq
from scipy.signal import detrend, get_window, welch
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst_efd_client.efd_helper import merge_packed_time_series


# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

# Create an EFD client
client = makeEfdClient()

# Global variables definition
sensorNames = [
    "SST top end ring +x -y",
    "SST top end ring -x -y",
    "SST spider spindle",
    "SST M2 surrogate",
]
sensorNamesShort = ["ter_pxmy", "ter_mxmy", "spider", "m2surr"]

# Create a folder for plots
os.makedirs("./plots", exist_ok=True)